# 5c. Parameter learning using Expectation Maximization (EM)
This notebook shows how parameter estimation is implemented in Thomas.

In [1]:
%run '_preamble.ipynb'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

available imports:
  import os
  import logging
  import pandas as pd
  import numpy as np

connect to this kernel with:
  jupyter console --existing 3998c6e9-9735-4ecf-92c4-60dda6be3e51

Logging to: "/Users/melle/software-development/thomas-master/logs/5c. Parameter learning using Expectation Maximization (EM).log"
Current date/time: 27-06-2020, 15:11
Current working directory: "/Users/melle/software-development/thomas-master/notebooks"


In [2]:
import functools

from thomas.core import examples
from thomas.core import factor
from thomas.core import BayesianNetwork, Factor, CPT, JPT
from thomas.core.bayesian_network import DiscreteNetworkNode
from thomas.jupyter import BayesianNetworkWidget

from IPython.display import display, HTML

## Initialization

### Load the BN

In [3]:
bn = examples.get_example17_3_network()

[n.cpt for n in bn.nodes.values()]

[P(A)
 A 
 a1    0.2
 a2    0.8
 dtype: float64,
 P(B|A)
 A   B 
 a1  b1    0.75
     b2    0.25
 a2  b1    0.10
     b2    0.90
 dtype: float64,
 P(C|A)
 A   C 
 a1  c1    0.50
     c2    0.50
 a2  c1    0.25
     c2    0.75
 dtype: float64,
 P(D|B)
 B   D 
 b1  d1    0.2
     d2    0.8
 b2  d1    0.7
     d2    0.3
 dtype: float64]

In [4]:
widget = BayesianNetworkWidget(bn)
widget

BayesianNetworkWidget(marginals_and_evidence={'marginals': {'A': {'a1': 0.2, 'a2': 0.8}, 'B': {'b1': 0.2299999…

### Load the data

In [5]:
filename = thomas.core.get_pkg_filename('dataset_17_3.csv')
df = pd.read_csv(filename, sep=';').set_index('Case')

print(f'df.shape: {df.shape[0]} rows x {df.shape[1]} cols')
print(f'This dataset has {df.isna().sum().sum()} NAs')

df

df.shape: 5 rows x 4 cols
This dataset has 8 NAs


,A,B,C,D
Case,,,,
d1,NaN,b1,c2,NaN
d2,NaN,b1,NaN,d2
d3,NaN,b2,c1,d1
d4,NaN,b2,c1,d1
d5,NaN,b1,NaN,d2


In [6]:
df.fillna('NaN').groupby(bn.scope, observed=True).size().reset_index().replace('NaN', np.nan)

,A,B,C,D,0
0,NaN,b1,NaN,d2,2
1,NaN,b1,c2,NaN,1
2,NaN,b2,c1,d1,2


## EM Learning using empirical distribution

In [7]:
# Complete each row with missing data
expanded = list(map(lambda x: bn.complete_case(x[1], include_weights=True), df.iterrows()))
expanded

[    A   B   C   D    weight
 0  a1  b1  c2  d1  0.111111
 1  a1  b1  c2  d2  0.444444
 2  a2  b1  c2  d1  0.088889
 3  a2  b1  c2  d2  0.355556,
     A   B   C   D    weight
 0  a1  b1  c1  d2  0.326087
 1  a1  b1  c2  d2  0.326087
 2  a2  b1  c1  d2  0.086957
 3  a2  b1  c2  d2  0.260870,
     A   B   C   D    weight
 0  a1  b2  c1  d1  0.121951
 1  a2  b2  c1  d1  0.878049,
     A   B   C   D    weight
 0  a1  b2  c1  d1  0.121951
 1  a2  b2  c1  d1  0.878049,
     A   B   C   D    weight
 0  a1  b1  c1  d2  0.326087
 1  a1  b1  c2  d2  0.326087
 2  a2  b1  c1  d2  0.086957
 3  a2  b1  c2  d2  0.260870]

In [8]:
# Sum the results of the cases
summed = pd.concat(expanded).groupby(bn.scope).sum()

# Normalize and select the column (--> pd.Series)
summed = (summed / summed.sum())['weight']

# Show output
summed

A   B   C   D 
a1  b1  c1  d2    0.130435
        c2  d1    0.022222
            d2    0.219324
    b2  c1  d1    0.048780
a2  b1  c1  d2    0.034783
        c2  d1    0.017778
            d2    0.175459
    b2  c1  d1    0.351220
Name: weight, dtype: float64

In [9]:
## Add the result to a Factor of all zeroes, to ensure all possible combinations of 
# variables are included in the JPT.
try:
    Pr_D = Factor.from_series(summed)
except:
    print('Oh .. previous version, eh?')
    zeroes = Factor(0, variable_states=bn.states)
    Pr_D = Factor((zeroes._data + summed).fillna(0))
    
Pr_D

factor(A,B,C,D)
A   B   C   D 
a1  b1  c1  d1    0.000
            d2    0.130
        c2  d1    0.022
            d2    0.219
    b2  c1  d1    0.049
            d2    0.000
        c2  d1    0.000
            d2    0.000
a2  b1  c1  d1    0.000
            d2    0.035
        c2  d1    0.018
            d2    0.175
    b2  c1  d1    0.351
            d2    0.000
        c2  d1    0.000
            d2    0.000
dtype: float64

Using the emperical distribution `Pr_D`, we can compute the updated CPTs as illustrated below.

In [10]:
# Compute P(A)
Pr_D.project('A')

factor(A)
A 
a1    0.421
a2    0.579
dtype: float64

In [11]:
# Compute P(B|A)
Pr_D.project(['B','A']) / Pr_D.project(['A'])

factor(A,B)
A   B 
a1  b1    0.884
    b2    0.116
a2  b1    0.394
    b2    0.606
dtype: float64

In [12]:
# Compute P(C|A)
Pr_D.project(['C','A']) / Pr_D.project(['A'])

factor(A,C)
A   C 
a1  c1    0.426
    c2    0.574
a2  c1    0.666
    c2    0.334
dtype: float64

In [13]:
# Compute P(B|D)
Pr_D.project(['B','D']) / Pr_D.project(['B'])

factor(B,D)
B   D 
b1  d1    0.067
    d2    0.933
b2  d1    1.000
    d2    0.000
dtype: float64

## EM Learning the using junction tree algorithm

In [14]:
bn = examples.get_example17_3_network()

In [15]:
# For reference, these are the CPTs before we do anything.
CPTs = [n.cpt for n in bn.nodes.values()]
CPTs

[P(A)
 A 
 a1    0.2
 a2    0.8
 dtype: float64,
 P(B|A)
 A   B 
 a1  b1    0.75
     b2    0.25
 a2  b1    0.10
     b2    0.90
 dtype: float64,
 P(C|A)
 A   C 
 a1  c1    0.50
     c2    0.50
 a2  c1    0.25
     c2    0.75
 dtype: float64,
 P(D|B)
 B   D 
 b1  d1    0.2
     d2    0.8
 b2  d1    0.7
     d2    0.3
 dtype: float64]

In [16]:
bn['A'].cpt

A,a1,a2
,0.2,0.8


In [17]:
widget = BayesianNetworkWidget(bn)
widget

BayesianNetworkWidget(marginals_and_evidence={'marginals': {'A': {'a1': 0.2, 'a2': 0.8}, 'B': {'b1': 0.2299999…

In [18]:
copy = bn.copy()
[n.cpt for n in copy.nodes.values()]

[P(A)
 A 
 a1    0.2
 a2    0.8
 dtype: float64,
 P(B|A)
 A   B 
 a1  b1    0.75
     b2    0.25
 a2  b1    0.10
     b2    0.90
 dtype: float64,
 P(C|A)
 A   C 
 a1  c1    0.50
     c2    0.50
 a2  c1    0.25
     c2    0.75
 dtype: float64,
 P(D|B)
 B   D 
 b1  d1    0.2
     d2    0.8
 b2  d1    0.7
     d2    0.3
 dtype: float64]

In [19]:
bn.P('A|B=b2,D=d1')

A,a1,a2
,0.064935,0.935065


In [20]:
copy.EM_learning(df, max_iterations=1)

[n.cpt for n in copy.nodes.values()]

[P(A)
 A 
 a1    0.421
 a2    0.579
 dtype: float64,
 P(B|A)
 A   B 
 a1  b1    0.884
     b2    0.116
 a2  b1    0.394
     b2    0.606
 dtype: float64,
 P(C|A)
 A   C 
 a1  c1    0.426
     c2    0.574
 a2  c1    0.666
     c2    0.334
 dtype: float64,
 P(D|B)
 B   D 
 b1  d1    0.067
     d2    0.933
 b2  d1    1.000
     d2    0.000
 dtype: float64]